<a href="https://colab.research.google.com/github/Cirediallo/DLIL/blob/main/VoiD_with_PySpark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Installing SPARK for Playing


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b9b247e9c26c2f9521d6f6d98368e68b8832ed7b05a5b31c16eee51318a4923d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install -q findspark
import findspark
findspark.init()


# SPARK Installed... Start playing

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
!wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
!ls


sample_data  small_page_links.nt


In [5]:
lines = spark.read.text("small_page_links.nt").rdd.map(lambda r: r[0])


In [6]:
lines.take(5)

['<http://dbpedia.org/resource/AfghanistanHistory> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/History_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AfghanistanGeography> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Geography_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AccessibleComputing> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Computer_accessibility> .',
 '<http://dbpedia.org/resource/AfghanistanPeople> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Demography_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AfghanistanCommunications> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Communications_in_Afghanistan> .']

In [7]:
lines.count()

100000

In [12]:
import re
def parseTriple(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    return parts[0], parts[1], parts[2]
triples=lines.map(lambda x:parseTriple(x))
triples.count()
triples.take(1)

[('<http://dbpedia.org/resource/AfghanistanHistory>',
  '<http://dbpedia.org/property/wikilink>',
  '<http://dbpedia.org/resource/History_of_Afghanistan>')]

# Exercice: Computing the VoiD for the Nt File with PySpark RDD

Write RDD PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

- QB1 CONSTRUCT { <D> v:triples ?x }
WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

- QB2 CONSTRUCT { <D> v:classes ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

- QB3 CONSTRUCT { <D> v:properties ?x }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

- QB4 CONSTRUCT { <D> v:distinctSubjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

- QB5 CONSTRUCT { <D> v:distinctObjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [17]:
triples.count()
# this one work too
#triples.map(lambda x: len(x)).count()

100000

QB1

In [29]:
print(triples.map(lambda x: x[0]).distinct().count())
print(triples.map(lambda x: x[1]).distinct().count())
print(triples.map(lambda x: x[2]).distinct().count())

800
1
61510


QB2

In [45]:
triples.filter(lambda x: x[2] ==  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>').map(lambda x: x[0]).distinct().count()
#triples.filter(lambda x: x[2] ==  'a').map(lambda x: x[0]).distinct().count()


0

QB3

In [23]:
triples.map(lambda x: x[2]).distinct().count()


61510

- QC1 CONSTRUCT { <D> v:classPartition [ v:class ?c ] } WHERE { ?s a ?c }

- QC2 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:triples ?x ] }
WHERE { SELECT (COUNT(?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }
- QC3 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:classes ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?d) AS ?x) ?c WHERE { ?s a ?c , ?d } GROUP BY ?c }
- QC4 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:properties ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c } 
- QC5 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c WHERE { ?s a ?c } GROUP BY ?c }
- QC6 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }

QC2

In [43]:
print(triples.filter(lambda x: x[1] == '<http://dbpedia.org/property/wikilink>').join(triples).take(5))
print(triples.filter(lambda x: x[1] == '<http://dbpedia.org/property/wikilink>').join(triples).groupByKey().count())


[('<http://dbpedia.org/resource/AfghanistanHistory>', ('<http://dbpedia.org/property/wikilink>', '<http://dbpedia.org/property/wikilink>')), ('<http://dbpedia.org/resource/AfghanistanPeople>', ('<http://dbpedia.org/property/wikilink>', '<http://dbpedia.org/property/wikilink>')), ('<http://dbpedia.org/resource/AfghanistanCommunications>', ('<http://dbpedia.org/property/wikilink>', '<http://dbpedia.org/property/wikilink>')), ('<http://dbpedia.org/resource/AfghanistanTransportations>', ('<http://dbpedia.org/property/wikilink>', '<http://dbpedia.org/property/wikilink>')), ('<http://dbpedia.org/resource/AssistiveTechnology>', ('<http://dbpedia.org/property/wikilink>', '<http://dbpedia.org/property/wikilink>'))]
800


In [ ]:
#toto = triples.filter(lambda x: x[1] ==  'ns#type')
#joined = toto.join(triples)
#joined.take(5)


In [47]:
print(triples.filter(lambda x: x[1] == '<http://dbpedia.org/property/wikilink>').join(triples).groupBy(lambda x: x[1]).take(5))

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

QC3

- QD1 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ] } WHERE { ?s ?p ?o }
- QD2 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:triples ?x ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD3 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD4 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }

In [ ]:
Your code ??

- QE1 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:propertyPartition [ v:property ?p ] ] }
WHERE { ?s a ?c ; ?p ?o }
- QE2 CONSTRUCT { <D> v:classPartition [ v:class ?c
v:propertyPartition [ v:property ?p ; v:triples ?x ] ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE3 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctSubjects ?x ] ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE4 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctObjects ?x ; v:property ?p ] ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }

In [ ]:
Your code

# Same Exercice with PySpark SQL



In [ ]:
from pyspark.sql import Row
spo=triples.map(lambda x: Row(subject=x[0],predicate=x[1],object=x[2]))
df=spark.createDataFrame(spo)
df.createOrReplaceTempView("SPO")
df.show()



+--------------------+--------------------+--------------------+
|             subject|           predicate|              object|
+--------------------+--------------------+--------------------+
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<ht

In [ ]:

result=spark.sql("select subject from SPO")
result.show()

+--------------------+
|             subject|
+--------------------+
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
+--------------------+
only showing top 20 rows



Write SQL PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

QB1 CONSTRUCT { v:triples ?x } WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

QB2 CONSTRUCT { v:classes ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

QB3 CONSTRUCT { v:properties ?x } WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

QB4 CONSTRUCT { v:distinctSubjects ?x } WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

QB5 CONSTRUCT { v:distinctObjects ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [ ]:
Your Code